In [ ]:
from google.colab import files

uploaded = files.upload()

Saving train_data.csv to train_data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('train_data.csv', sep=';')

In [ ]:
!pip install xgboost


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df.drop(['was_alarm_event'], axis=1)
y = df['was_alarm_event']

In [ ]:
X = X.drop(columns=['Unnamed: 0'], axis=1)

In [ ]:
X.columns

Index(['day_datetime_timestamp', 'day_tempmax', 'day_tempmin', 'day_temp',
       'hour_temp', 'day_dew', 'day_humidity', 'day_precip', 'hour_precipprob',
       'day_precipcover', 'hour_snow', 'hour_windgust', 'hour_windspeed',
       'hour_winddir', 'hour_pressure', 'hour_cloudcover', 'hour_visibility',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_sunrise',
       'day_sunset', 'day_moonphase', 'weather_Clear', 'russian', 'ukrainian',
       'forces', 'ukraine', 'russia', 'oblast', 'military', 'operations',
       'offensive', 'bakhmut', 'likely', 'war', 'defense', 'reported',
       'claimed', 'stated', 'continued', 'kremlin', 'putin', 'wagner',
       'president', 'general', 'city', 'donetsk', 'western'],
      dtype='object')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor', gpu_id=0)

param_dist = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0.8, 1.0, 1.2]
}

random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=50, scoring='accuracy', n_jobs=-1, cv=3, verbose=2)

random_search.fit(X_train, y_train)


y_pred = random_search.best_estimator_.predict(X_test)

print(classification_report(y_test, y_pred))



Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:49:49] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:49:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:49:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:49:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training,

              precision    recall  f1-score   support

           0       0.84      0.92      0.88    100620
           1       0.75      0.57      0.65     39750

    accuracy                           0.82    140370
   macro avg       0.80      0.75      0.76    140370
weighted avg       0.82      0.82      0.82    140370



In [ ]:
from google.colab import files

files.download('best_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle

with open('best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
